### Stegmatation and Counters

In [6]:
import cv2
import numpy as np

def _drawContour(img):
    cv2.imshow('original %s'%img,img)
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray_img,30,170)
    contours,hierachy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img,contours,-1,(0,255,0),3)
    cv2.imshow('original with Counters %s'%img,img)

img = cv2.imread('./images/table.jpg')
img = cv2.resize(img,None,fx=0.2,fy=0.2)
_drawContour(img)

img2 = cv2.imread('./images/input____.jpg')
_drawContour(img2)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Sort By Area

In [16]:
import cv2
import numpy as np

img = cv2.imread('./images/shapes.jpg')
img = cv2.resize(img,None,fx=0.3,fy=0.3)
bg = np.zeros((img.shape[0],img.shape[1],3))
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
edged = cv2.Canny(gray_img,30,170)
contours,hierachy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

def get_contours_area(contours):
    areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areas.append(area)
    return areas

print "before sorting : " + str(get_contours_area(contours))

sorted_contours = sorted(contours , key=cv2.contourArea, reverse=True)
print "after sorting : " + str(get_contours_area(sorted_contours))

for i,cnt in enumerate(sorted_contours):
    cv2.drawContours(img , [cnt], -1 , (255,0,0) , 3)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(img,str(i+1),(cx-10,cy+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
    cv2.waitKey(0)
    cv2.imshow("sorted",img)

cv2.waitKey(0)
cv2.destroyAllWindows()

before sorting : [69076.0, 53820.5, 51702.0, 46929.5, 73980.5, 68367.0]
after sorting : [73980.5, 69076.0, 68367.0, 53820.5, 51702.0, 46929.5]


### Sort From left to Right

In [172]:
import cv2
import numpy as np

img = cv2.imread('./images/shapes.jpg')
img = cv2.resize(img,None,fx=0.3,fy=0.3)
original_img = img.copy()
bg = np.zeros((img.shape[0],img.shape[1],3))
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
edged = cv2.Canny(gray_img,30,170)
contours,hierachy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

def x_cord_contours(contours):
    if cv2.contourArea(contours) > 10:
        M = cv2.moments(contours)
        return int(M['m10']/M['m00'])
    
def label_contour_center(image,cnt,i):
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.circle(image,(cx,cy),10,(0,0,0),-1)
    return image
    
for(i,cnt) in enumerate(contours):
    labled_img = label_contour_center(img,cnt,i)
    
#cv2.imshow("labled",labled_img)
#cv2.waitKey(0)

counters_left_to_right = sorted(contours , key = x_cord_contours , reverse = False)

for (i,c) in enumerate(counters_left_to_right):
    cv2.drawContours(original_img,[c],-1,(0,0,0),3)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(original_img,str(i+1),(cx-10,cy+10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
    cv2.imshow('texted',original_img)
    cv2.waitKey(0)
    (x,y,w,h) = cv2.boundingRect(c)
    
    cropped_contour = original_img[y:y+h , x:x+w]
    image_name = "output_shape_number"+str(i+1)+".jpg"
    print image_name
    cv2.imwrite(image_name , cropped_contour)

cv2.destroyAllWindows()

output_shape_number1.jpg
output_shape_number2.jpg
output_shape_number3.jpg
output_shape_number4.jpg
output_shape_number5.jpg
output_shape_number6.jpg


### Approximation Contours

In [171]:
import cv2
import numpy as np

img = cv2.imread('./images/home.jpg')
cv2.imshow('Original House',img)
cv2.waitKey(0)
o_img = img.copy()

gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
ret , thresh = cv2.threshold(gray , 127,255,cv2.THRESH_BINARY_INV)

contours,hierarchy = cv2.findContours(thresh.copy() , cv2.RETR_LIST , cv2.CHAIN_APPROX_NONE)

for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(o_img,(x,y),(x+w,y+h),(0,0,255),2)
    cv2.imshow('Bounding Rect House',o_img)
cv2.waitKey(0)

for c in contours:
    accuracy = 0.02 * cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,accuracy,True)
    cv2.drawContours(img,[approx],0,(0,255,0),2)
    cv2.imshow('Approx House',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Convex Hull

In [181]:
import cv2
import numpy as np

img = cv2.imread('./images/hand.jpg')
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

ret,thresh = cv2.threshold(gray_img,176,255,0)
contours,hierachy = cv2.findContours(thresh.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)

n = len(contours)-1
contours = sorted(contours,key=cv2.contourArea,reverse=False)[:n]

for c in contours:
    hull = cv2.convexHull(c)
    cv2.drawContours(img,[hull],0,(0,255,0),2)
    cv2.imshow('Convex Hull',img)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Matches

In [182]:
import cv2
import numpy as np
import pyttsx

target = cv2.imread('./images/shapes.jpg')
target = cv2.resize(target,None,fx=0.3,fy=0.3)
target_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)
template = cv2.imread('./images/template.jpg')
template_gray = cv2.cvtColor(target,cv2.COLOR_BGR2GRAY)

_,thresh1 = cv2.threshold(template_gray,127,255,0)
_,thresh2 = cv2.threshold(target_gray,127,255,0)

contours , hierachy = cv2.findContours(thresh1,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
sorted_contours = sorted(contours,key=cv2.contourArea,reverse=True)
template_contour = contours[1]

contours , hierachy = cv2.findContours(thresh2,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    match = cv2.matchShapes(template_contour,c,1,0.0)
    cv2.drawContours(template_gray,[c],-1,(0,255,0),3)        
    if match < 0.15:
        closet_contour = c
    else:
        closet_contour = []
        
engine = pyttsx.init()
engine.say('Okay i think the perdect match of the shape is this')
cv2.imshow('Voila',target)
cv2.drawContours(target,[closet_contour],-1,(0,0,0),15)        
cv2.imshow('Template',template)
engine.runAndWait()
cv2.imshow('Voila',target)


cv2.waitKey(0)
cv2.destroyAllWindows()

### Identify Shapes

In [71]:
import cv2
import numpy
import pyttsx

def drawIt(img,cnt,shape_name):
    cv2.drawContours(img,[cnt],-1,(0,0,0),3)
    M = cv2.moments(cnt)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.putText(img,shape_name,(cx-50,cy),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0) ,1)
    cv2.imshow("img",img)
    engine.say("this is an " + shape_name)
    engine.runAndWait()

img = cv2.imread('./images/shapes.jpg')
img = cv2.resize(img,None,fx=0.3,fy=0.3)
show_im = cv2.resize(img,None,fx=0.7,fy=0.7)
cv2.imshow("original Image",show_im)

gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
edged = cv2.Canny(gray,200,255)

ret,thresh = cv2.threshold(edged,1,2,0)

contours , hierachy = cv2.findContours(thresh.copy() , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_NONE)

engine = pyttsx.init()

for cnt in contours:
    approx = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
    if len(approx) == 3:
        shape_name = "Triangle"
        drawIt(img,cnt,shape_name)
    elif len(approx) == 4:
        (x,y,w,h) = cv2.boundingRect(cnt)
        if abs(w-h) <= 1:
            shape_name = "square"
        else:
            shape_name = "rectangle"
        drawIt(img,cnt,shape_name)
    elif len(approx) == 13:
        shape_name = "ellipse"
        drawIt(img,cnt,shape_name)
    elif len(approx) == 16:
        shape_name = "Circle"
        drawIt(img,cnt,shape_name)
    elif len(approx) == 5:
        shape_name = "Polygon"
        drawIt(img,cnt,shape_name)
    
cv2.imshow("img",img)
cv2.destroyAllWindows()

### Line Detection
##### Using HoughLines

In [241]:
import cv2
import numpy as np
import pyttsx

img = cv2.imread('./images/sudoku.jpg')
cv2.imshow('Original img',img)
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray_img, 100,170, apertureSize=3)

lines = cv2.HoughLines(edges,1,np.pi/ 180 , 240)

for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0+1000*(-b))
    y1 = int(y0+1000*(a))
    x2 = int(x0-1000*(-b))
    y2 = int(y0-1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)

cv2.imshow('Hough Lines',img)
engine = pyttsx.init()
engine.say("i found "+str(len(lines[0])/2)+"line")
engine.runAndWait()
cv2.destroyAllWindows()

##### using Probalistic Hough Lines

In [3]:
import cv2
import numpy as np

img = cv2.imread('./images/sudoku.jpg')
cv2.imshow('Original img',img)
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray_img, 100,170, apertureSize=3)

lines = cv2.HoughLinesP(edges,1,np.pi/ 180 , 10,5,10)

for x1,y1,x2,y2 in lines[0]:
    cv2.line(img, (x1,y1),(x2,y2),(0,255,0),3)

cv2.imshow('Probalistic Hough Lines',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Circle Image Detection

In [7]:
import cv2
import numpy as np

img = cv2.imread('./images/bottles_caps.jpg')
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
blur = cv2.medianBlur(gray_img, 5)

circles = cv2.HoughCircles(blur, cv2.cv.CV_HOUGH_GRADIENT, 1.5, 10)

for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(img,(i[0], i[1]), i[2], (255, 0, 0), 2)
    # draw the center of the circle
    cv2.circle(img, (i[0], i[1]), 2, (0, 255, 0), 5)

cv2.imshow('detected circles', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Blobs Dector

In [25]:
import cv2
import numpy as np

img = cv2.imread('./images/blobs2.jpg')
gray_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

detector = cv2.SimpleBlobDetector()
keypoints = detector.detect(gray_img)

blank = np.zeros((1,1))
blobs = cv2.drawKeypoints(img,keypoints,blank,(0,255,255),
                          cv2.DRAW_MATCHES_FLAGS_DEFAULT)

cv2.imshow("Blobs",blobs)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Blobs And Circle identifier -Mini Project- 

In [70]:
import cv2
import numpy as np
import pyttsx as px

img = cv2.imread('./images/blobsAndCircles.jpg',0)
cv2.imshow('original',img)

engine = px.init()

detector = cv2.SimpleBlobDetector()

keypoints = detector.detect(img)

blank = np.zeros((1,1))

blobs = cv2.drawKeypoints(img,keypoints,blank,(0,0,255),
                         cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
height = img.shape[1]
number_of_blobs = len(keypoints)
text = "Total Number of Blobs is "+str(len(keypoints))
cv2.putText(blobs,text,(20,height),cv2.FONT_HERSHEY_SIMPLEX,1,(100,0,255),2)
cv2.imshow('Blobs using Default Detector',blobs)
engine.say(text)
engine.runAndWait()

params = cv2.SimpleBlobDetector_Params()

params.filterByArea = True
params.minArea = 100

params.filterByCircularity = True
params.minCircularity = 0.9

params.filterByConvexity = False
params.minConvexity = 0.2

params.filterByInertia = True
params.minInertiaRatio = 0.01

detector = cv2.SimpleBlobDetector(params)

keypoints = detector.detect(img)

blank = np.zeros((1,1))

blobs = cv2.drawKeypoints(img,keypoints,blank,(0,0,255),
                         cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
height = img.shape[1]
number_of_blobs = len(keypoints)
text = "Blobs with circular shape is "+str(len(keypoints))
cv2.putText(blobs,text,(0,height),cv2.FONT_HERSHEY_SIMPLEX,1,(100,0,255),2)
cv2.imshow('Blobs with circular shape',blobs)
engine.say(text)
engine.runAndWait()


cv2.waitKey(0)
cv2.destroyAllWindows()